In [7]:
import pandas as pd
from rdkit import Chem

df = pd.read_csv('/opt/home/revoli/eva/Interformer/result/5qay_docked_infer_ensemble.csv')

ligands = Chem.SDMolSupplier('/opt/home/revoli/eva/Interformer/jane_VS/infer/5qay_docked.sdf')

rank_list = df['pose_rank'].unique().tolist()

df

,Target,pose_rank,uff_pose_rank,Molecule ID,pIC50,pred_pIC50,pred_pose,pred_pIC50_var
0,5qay,0,0,Vonoprazan,0.0,5.473633,0.084611,1.064447
1,5qay,1,0,Vonoprazan,0.0,4.998047,0.175426,1.429092
2,5qay,2,0,Vonoprazan,0.0,6.080078,0.511797,0.766464
3,5qay,3,0,Vonoprazan,0.0,5.244141,0.252014,1.141535
4,5qay,4,0,Vonoprazan,0.0,4.442383,0.016093,0.402297
...,...,...,...,...,...,...,...,...
120307,5qay,126788,0,Cefozopran (hydrochloride),0.0,3.462402,0.021569,0.110195
120308,5qay,126789,0,Cefozopran (hydrochloride),0.0,1.458984,0.000088,0.251374
120309,5qay,126790,0,Cefozopran (hydrochloride),0.0,4.556641,0.085164,0.696762
120310,5qay,126791,0,Cefozopran (hydrochloride),0.0,2.467773,0.000650,0.018724


In [8]:
from tqdm import tqdm

penalties = []
for rank in tqdm(rank_list):
    penalty = float(ligands[rank].GetProp('r_epik_State_Penalty'))
    penalties.append(penalty)
df['epik_penalty'] = penalties

df.to_csv('/tmp/5qay_Interformer.csv', index=False)
df

 66%|██████▌   | 79157/120312 [00:34<00:20, 1961.74it/s][16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[16:56:07] Warning: molecule is tagged as 2D, bu

,Target,pose_rank,uff_pose_rank,Molecule ID,pIC50,pred_pIC50,pred_pose,pred_pIC50_var,epik_penalty
0,5qay,0,0,Vonoprazan,0.0,5.473633,0.084611,1.064447,0.0025
1,5qay,1,0,Vonoprazan,0.0,4.998047,0.175426,1.429092,0.0025
2,5qay,2,0,Vonoprazan,0.0,6.080078,0.511797,0.766464,0.0025
3,5qay,3,0,Vonoprazan,0.0,5.244141,0.252014,1.141535,0.0025
4,5qay,4,0,Vonoprazan,0.0,4.442383,0.016093,0.402297,0.0025
...,...,...,...,...,...,...,...,...,...
120307,5qay,126788,0,Cefozopran (hydrochloride),0.0,3.462402,0.021569,0.110195,0.0000
120308,5qay,126789,0,Cefozopran (hydrochloride),0.0,1.458984,0.000088,0.251374,0.0000
120309,5qay,126790,0,Cefozopran (hydrochloride),0.0,4.556641,0.085164,0.696762,0.0000
120310,5qay,126791,0,Cefozopran (hydrochloride),0.0,2.467773,0.000650,0.018724,0.0000


In [11]:
len(df[df['epik_penalty'] < 0.5])

74346